In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore', np.RankWarning)

# Basic Heston model

paper: https://frouah.com/finance%20notes/Euler%20and%20Milstein%20Discretization.pdf <br>
https://quantpy.com.au/stochastic-volatility-models/simulating-heston-model-in-python/

In [5]:
def heston_samples(S0, T, N, M, r=0.02, rho=0.7, kappa=3, theta=0.04, sigma=0.6, v0=0.0625):
    dt = T/N
    S = np.full(shape=(M,N+1), fill_value=S0)
    v = np.full(shape=(M,N+1), fill_value=v0)
    for i in range(N):
        Z1 = np.random.normal(0, 1, M)
        Z2 = np.random.normal(0, 1, M)
        Zv = Z1
        Zs = rho*Z1 + np.sqrt(1-rho**2)*Z2
        v[:,i+1] = np.fmax(v[:,i] + kappa*(theta-v[:,i])*dt + sigma*np.sqrt(v[:,i]*dt)*Zv, 0)
        S[:,i+1] = S[:,i] * np.exp((r - 0.5*v[:,i])*dt + np.sqrt(v[:,i]*dt)*Zs)
    return S, v

## LongstaffSchwartz Pricing

In [8]:
def longstaff_schwartz(T, r, stockprice, exercise_dates, K, dimension):
    M = stockprice.shape[0]
    J = stockprice.shape[1] - 1
    N = exercise_dates
    dt = T / N
    n = J // N  # discretizing steps per period
    
    if n * N != J:
        raise Exception("Error: number of exercise dates and number of steps per period do not match!")
    
    S = stockprice[:,::int(n)] 
    payoff = np.fmax(K-S, 0)
    stop = np.full(M, N)
    #itm0 = np.where(payoff[:,N] > 0)[0]
    #stop[itm0] = N
    cf = np.zeros((M, N+1))
    cont_array = np.zeros((M,N+1))
    stop_array = np.full(shape=(M,N+1), fill_value = 0)
    coeff_array = []
    for i in range(N, 0, -1):
        itm = np.where(payoff[:,i-1] > 0)[0]
        if len(itm) == 0:
            pass
        else:
            x = S[:,i-1][itm]
            y = payoff[itm, stop[itm]] * np.exp(-r * dt * (stop[itm]-i+1))
            coeffs = np.polyfit(x, y, dimension)
            coeff_array.insert(0,coeffs)
            cont = np.zeros(M)
            cont[itm] = np.polyval(coeffs, x) 
            cont_array[itm, i-1] = np.polyval(coeffs, x) 
            exercise = payoff[:,i-1]
            early_ex = np.where(exercise >= cont)[0]
            intersect = np.intersect1d(itm, early_ex)
            stop[intersect] = i-1
            stop_array[intersect, i-1] = 1 
    #paths = np.where(stop < np.inf)[0]
    #ls = [int(i) for i in stop[paths]]
    disc_payoff = payoff[np.arange(M), stop] * np.exp(-r * dt * stop)
    p = np.mean(disc_payoff)
    mc_err = 1.96 * np.std(disc_payoff) / np.sqrt(M)
    return np.round(p, 4), disc_payoff, stop, cont_array, stop_array, coeff_array #"Monte Carlo Error: {:%}".format(mc_err)

In [9]:
def longstaff_schwartz_multivariate(T, r, stockprice, volatility, exercise_dates, K, dimension):
    M = stockprice.shape[0]
    J = stockprice.shape[1] - 1
    N = exercise_dates
    dt = T / N
    n = J // N  # discretizing steps per period
    
    if n * N != J:
        raise Exception("Error: number of exercise dates and number of steps per period do not match!")
    
    S = stockprice[:,::int(n)] 
    v = volatility[:,::int(n)]
    payoff = np.fmax(K-S, 0)
    stop = np.full(M, N)

    cf = np.zeros((M, N+1))
    cont_array = np.zeros((M,N+1))
    stop_array = np.full(shape=(M,N+1), fill_value = 0)
    coeff_array = []
    for i in range(N, 0, -1):
        itm = np.where(payoff[:,i-1] > 0)[0]
        if len(itm) == 0:
            pass
        else:
            x1 = S[:,i-1][itm]
            x2 = v[:,i-1][itm]
            y = payoff[itm, stop[itm]] * np.exp(-r * dt * (stop[itm]-i+1))
            j = len(itm)
            if dimension == 'm1':
                X_1 = np.vstack((np.ones(j), x1, x2, x1**2, x1*x2))
            if dimension == 'm2':
                X_1 = np.vstack((np.ones(j), x1, x2, x1**2, x1**3, x1**4, x1*x2, x1**2 * x2))
            if dimension == 'm3':
                X_1 = np.vstack((np.ones(j), x1, x2, x1**2, x1*x2, x1**2 * x2, x1**3))

            X = X_1.T #transform
            regModel = LinearRegression()
            regModel.fit(X, y) # fit the model

            # get expected continuation values
            cont = np.zeros(M)
            cont[itm] = regModel.predict(X) 
            cont_array[itm, i-1] = regModel.predict(X)  
            
            exercise = payoff[:,i-1]
            early_ex = np.where(exercise >= cont)[0]
            intersect = np.intersect1d(itm, early_ex)
            stop[intersect] = i-1
            stop_array[intersect, i-1] = 1 
    disc_payoff = payoff[np.arange(M), stop] * np.exp(-r * dt * stop)
    p = np.mean(disc_payoff)
    mc_err = 1.96 * np.std(disc_payoff) / np.sqrt(M)
    return np.round(p, 4), "Monte Carlo Error: {:%}".format(mc_err)

In [11]:
S0 = 100.0          
T = 1.0                
N = 500
M = 500000

r = 0.02 
kappa = 3              # rate of mean reversion of variance under risk-neutral dynamics
theta = 0.04        # long-term mean of variance under risk-neutral dynamics
v0 = 0.0625          # initial variance under risk-neutral dynamics
rho = 0.7              # correlation between returns and variances under risk-neutral dynamics
sigma = 0.6   

In [12]:
model = heston_samples(S0, T, N, M)

In [13]:
stockprices = model[0]
variance = model[1]

### Results

In [15]:
for d in range(1,6,1):
    print(d)
    for ex in [5,10,20,25,50]:
        print('exercise dates', ex, ': ', longstaff_schwartz(T, r, stockprices, ex, 110, d)[0])
    print('-------------------------')

1
exercise dates 5 :  13.883
exercise dates 10 :  13.8718
exercise dates 20 :  13.8419
exercise dates 25 :  13.8235
exercise dates 50 :  13.7856
-------------------------
2
exercise dates 5 :  13.9222
exercise dates 10 :  13.9412
exercise dates 20 :  13.9372
exercise dates 25 :  13.9289
exercise dates 50 :  13.9127
-------------------------
3
exercise dates 5 :  13.9385
exercise dates 10 :  13.962
exercise dates 20 :  13.9692
exercise dates 25 :  13.9687
exercise dates 50 :  13.9623
-------------------------
4
exercise dates 5 :  13.9393
exercise dates 10 :  13.9608
exercise dates 20 :  13.9713
exercise dates 25 :  13.9694
exercise dates 50 :  13.9703
-------------------------
5
exercise dates 5 :  13.9399
exercise dates 10 :  13.9591
exercise dates 20 :  13.9711
exercise dates 25 :  13.9714
exercise dates 50 :  13.9751
-------------------------


Premia: PutAmer/ MC_AM_Alfonsi_LongstaffSchwartz <br>
N Simulations: 500000, N Exercise Dates: [5, 10, 20, 25, 50], Dimension Approx: 10 <br>
Strike: 110 <br>
Price: 13.9872, 14.0338, 14.0255, 14.0434, 14.0474 <br>
Strike: 120 <br>
Price: 22.1177, 22.1897, 22.1962, 22.1956, 22.22<br>
Strike: 130 <br>
Price: 31.0203, 31.1327, 31.1504, 31.1475, 31.1777<br>
Strike: 140 <br>
Price: 40.3652, 40.4988, 40.5189, 40.5323, 40.5681<br>
Strike: 100 <br>
Price: 7.1822, 7.2030, 7.2145, 7.2249, 7.2271<br>
Strike: 90 <br>
Price: 2.5177, 2.5175, 2.5275, 2.5325, 2.5277<br>
Strike: 80 <br>
Price: 0.4578, 0.4572, 0.4576, 0.4585, 0.4588<br>
Strike: 70 <br>
Price: 0.03868, 0.03849, 0.03888, 0.03791, 0.03840<br>

In [22]:
us_premia = np.zeros((5,8))

In [23]:
us_premia[:,0] = [0.03868, 0.03849, 0.03888, 0.03791, 0.03840]
us_premia[:,1] = [0.4578, 0.4572, 0.4576, 0.4585, 0.4588]
us_premia[:,2] = [2.5177, 2.5175, 2.5275, 2.5325, 2.5277]
us_premia[:,3] = [7.1822, 7.2030, 7.2145, 7.2249, 7.2271]
us_premia[:,4] = [13.9872, 14.0338, 14.0255, 14.0434, 14.0474 ]
us_premia[:,5] = [22.1177, 22.1897, 22.1962, 22.1956, 22.22]
us_premia[:,6] = [31.0203, 31.1327, 31.1504, 31.1475, 31.1777]
us_premia[:,7] = [40.3652, 40.4988, 40.5189, 40.5323, 40.5681]

In [30]:
df1 = pd.DataFrame({'70': us_premia[:, 0], '80': us_premia[:, 1], 
             '90': us_premia[:, 2], '100': us_premia[:, 3],
             '110': us_premia[:, 4], '120': us_premia[:, 5],
             '130': us_premia[:, 6], '140': us_premia[:, 7]}, index=[5, 10, 20, 25, 50])

In [29]:
P = np.zeros((5,8))
exercise_dates = [5,10,20,25,50]
K = np.arange(70,150,10)
for k in range(len(K)):
    print(k)
    for i in range(len(exercise_dates)):
        print(i)
        P[i,k] = longstaff_schwartz_multivariate(T, r, stockprices, variance, exercise_dates[i], K[k], 'm2')[0]

In [31]:
# American Put 
df2 = pd.DataFrame({'70': P[:, 0], '80': P[:, 1], 
             '90': P[:, 2], '100': P[:, 3],
             '110': P[:, 4], '120': P[:, 5],
             '130': P[:, 6], '140': P[:, 7]}, index=[5, 10, 20, 25, 50])

In [33]:
df = pd.concat([df2, df1], axis=1)

In [36]:
df.columns = [['LongstaffSchwartz', 'LongstaffSchwartz', 'LongstaffSchwartz', 'LongstaffSchwartz', 'LongstaffSchwartz', 'LongstaffSchwartz', 'LongstaffSchwartz', 'LongstaffSchwartz',
               ' PutAmerPremia',' PutAmerPremia',' PutAmerPremia',' PutAmerPremia',' PutAmerPremia',' PutAmerPremia',' PutAmerPremia',' PutAmerPremia']
        ,['70', '80', '90', '100', '110', '120', '130', '140', '70', '80', '90',
       '100', '110', '120', '130', '140']]

In [57]:
df.round(decimals=2)

LongstaffSchwartz                                                \
                  70    80    90   100    110    120    130    140   
5               0.04  0.46  2.53  7.21  14.00  22.13  31.06  40.40   
10              0.04  0.47  2.55  7.23  14.04  22.19  31.13  40.50   
20              0.04  0.47  2.55  7.24  14.05  22.22  31.15  40.53   
25              0.04  0.47  2.55  7.25  14.06  22.21  31.17  40.55   
50              0.04  0.47  2.56  7.25  14.06  22.22  31.18  40.55   

    PutAmerPremia                                                
               70    80    90   100    110    120    130    140  
5            0.04  0.46  2.52  7.18  13.99  22.12  31.02  40.37  
10           0.04  0.46  2.52  7.20  14.03  22.19  31.13  40.50  
20           0.04  0.46  2.53  7.21  14.03  22.20  31.15  40.52  
25           0.04  0.46  2.53  7.22  14.04  22.20  31.15  40.53  
50           0.04  0.46  2.53  7.23  14.05  22.22  31.18  40.57

In [58]:
2.53/2.56 - 1

-0.011718750000000111

In [59]:
31.06/31.02 - 1

0.0012894906511926596

In [8]:
for k in range(70, 110, 10):
    print(k)
    for ex in [5,10,20,25,50]:
        print('exercise dates', ex, ': ', longstaff_schwartz_multivariate(T, r, stockprices, variance, ex, k, 'm2')[0])
    print('-------------------------')

70
exercise dates 5 :  0.039
exercise dates 10 :  0.0392
exercise dates 20 :  0.0393
exercise dates 25 :  0.0396
exercise dates 50 :  0.0397
-------------------------
80
exercise dates 5 :  0.4675
exercise dates 10 :  0.4698
exercise dates 20 :  0.47
exercise dates 25 :  0.4714
exercise dates 50 :  0.4716
-------------------------
90
exercise dates 5 :  2.541
exercise dates 10 :  2.5506
exercise dates 20 :  2.5584
exercise dates 25 :  2.5593
exercise dates 50 :  2.561
-------------------------
100
exercise dates 5 :  7.2158
exercise dates 10 :  7.2383
exercise dates 20 :  7.2498
exercise dates 25 :  7.2523
exercise dates 50 :  7.2578
-------------------------


In [9]:
for k in range(110, 150, 10):
    print(k)
    for ex in [5,10,20,25,50]:
        print('exercise dates', ex, ': ', longstaff_schwartz_multivariate(T, r, stockprices, variance, ex, k, 'm2')[0])
    print('-------------------------')

110
exercise dates 5 :  14.0057
exercise dates 10 :  14.0451
exercise dates 20 :  14.0592
exercise dates 25 :  14.0573
exercise dates 50 :  14.0658
-------------------------
120
exercise dates 5 :  22.1394
exercise dates 10 :  22.195
exercise dates 20 :  22.2143
exercise dates 25 :  22.2222
exercise dates 50 :  22.2285
-------------------------
130
exercise dates 5 :  31.0668
exercise dates 10 :  31.128
exercise dates 20 :  31.1595
exercise dates 25 :  31.1615
exercise dates 50 :  31.1804
-------------------------
140
exercise dates 5 :  40.4124
exercise dates 10 :  40.4953
exercise dates 20 :  40.5456
exercise dates 25 :  40.5454
exercise dates 50 :  40.5552
-------------------------


Premia: PutEur/ MC_Alfonsi <br>
500000 Iterations, 500 TimeStepNumber, KNUTH, Third Order for the CIR <br>
Price: 0.03803, 0.4538, 2.4959, 7.0840, 13.7074, 21.5707, 30.1584, 39.1755 

Premia: PutAmer / MC_AM_Alfonsi_LongstaffSchwartz <br>
500000 Iterations, KNUTH, Dimension 10 <br>
Strike: 70 <br>
Price: 0.03956, 0.4613, 2.5230

## European Put

In [41]:
def eur_put_price(T, r, M, S, K):
    payoffs = np.fmax(0, K - S[:,-1])
    return np.round(np.average(np.exp(-r*T) * payoffs), 5)

In [43]:
eur_put = []
for K in range(70,150,10):
    p = eur_put_price(T, r, M, stockprices, K)
    eur_put.append(p)
    print(K, p)

70 0.03787
80 0.46209
90 2.51381
100 7.10631
110 13.72719
120 21.59436
130 30.18631
140 39.20529


In [44]:
eur_pre = [0.03803, 0.4538, 2.4959, 7.0840, 13.7074, 21.5707, 30.1584, 39.1755]

Premia: PutEur/ MC_Alfonsi <br>
500000 Iterations, 500 TimeStepNumber, KNUTH, Third Order for the CIR <br>
Price: 0.03803, 0.4538, 2.4959, 7.0840, 13.7074, 21.5707, 30.1584, 39.1755 

In [52]:
def relative_error(x, x_approx):
    err = x_approx / x - 1
    return err # "Relative Error: {:%}".format(err)

In [53]:
rel_err = []
for i in range(8):
    err = relative_error(eur_pre[i], eur_put[i])
    rel_err.append(err)

In [54]:
data_eu = {'european put': eur_put, 'Premia PutEur': eur_pre, 'relative error': rel_err}

In [55]:
df_eur = pd.DataFrame(data_eu, index=np.arange(70,150,10))

In [56]:
df_eur

,european put,Premia PutEur,relative error
70,0.03787,0.03803,-0.004207
80,0.46209,0.45380,0.018268
90,2.51381,2.49590,0.007176
100,7.10631,7.08400,0.003149
110,13.72719,13.70740,0.001444
120,21.59436,21.57070,0.001097
130,30.18631,30.15840,0.000925
140,39.20529,39.17550,0.000760
